# openelec example walkthrough

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from pathlib import Path
import os

import pandas as pd
import geopandas as gpd
import folium

import matplotlib.pyplot as plt
from IPython.display import display, Markdown

from openelec.clustering import prepare_clusters
from openelec.local import LocalModel
from openelec import io

In [ ]:
country = 'Lesotho'

# Prepare clusters

In [ ]:
folder_input = Path.home() / 'Documents/GIS'
ghs_in = folder_input / 'GHS_POP_250.tif'
gdp_in = folder_input / 'GDP.tif'
travel_in = folder_input / 'Travel.tif'
ntl_in = folder_input / 'VIIRS_Africa_2016.tif'
aoi_in = folder_input / 'gadm.gpkg'
grid_in = folder_input / 'africagrid.gpkg'

folder_output = Path.home() / f'Code/openelec-api/openelec_api/static/{country.lower()}'
clusters_out = folder_output / 'clusters.geojson'

clusters = prepare_clusters(country,
                            ghs_in=ghs_in,
                            gdp_in=gdp_in,
                            travel_in=travel_in,
                            ntl_in=ntl_in,
                            aoi_in=aoi_in,
                            grid_in=grid_in,
                            clusters_out=clusters_out)   

# National

# Local

In [ ]:
buildings_in = Path('test_data/buildings.geojson')
output_data = Path('test_output')

In [ ]:
min_area = 0  # exclude any buildings with area (in m2) below this value
lm = LocalModel(data=buildings_in, min_area=min_area)

In [ ]:
m = folium.Map([lm.y_mean, lm.x_mean],
               zoom_start=15,
               control_scale=True)

popup_html = '<h2>Latitude: " + lat + "</h2><h2>Longitude: " + lng + "</h2><h3>Copy these values into the variables below.</h3>'
folium.ClickForMarker(popup=popup_html).add_to(m)
folium.GeoJson(lm.targets).add_to(m)
display(Markdown('### Please click on the desired location for the PV point.'))
display(Markdown('### Then click on the marker that appears and copy the values in below (with all decimals values!).'))
display(m)

In [ ]:
latitude = -29.5431
longitude = 28.1412

In [ ]:
lm.connect_targets(origin=(latitude, longitude))

In [ ]:
lm.parameters(demand=8,             # average demand in kWh/person/month
              tariff=0.2,          # USD/kWh charged
              gen_cost=4000,        # USD/kW for installation, excluding distribution network
              cost_wire=20,         # USD per metre
              cost_connection=140,  # USD per connection/node
              opex_ratio=0.02,      # % of capex per year (values above 1 are divided by 100)
              years=20,             # years over which to amortize (and maintain)
              discount_rate=0.06)   # discount rate (values above 1 are divided by 100)

In [ ]:
lm.model()

In [ ]:
lm.spatialise()

display(Markdown('### A small graph is shown below, with the PV point in green'))
fig, ax = plt.subplots(figsize=(10,10))
lm.network_gdf.loc[lm.network_gdf['enabled'] == 1].plot(ax=ax, color='red')
lm.buildings_gdf.plot(ax=ax, color='blue')
ax.scatter(longitude, latitude, s=80, lw=0, c='green')
plt.show()

In [ ]:
lm.save_to_path(output_data)
print('Results saved to', output_data.absolute())

In [ ]:
lm.summary()
display(Markdown(f'Total houses connected: {lm.results["connected"]} out of {len(lm.buildings_gdf)-1}'))
display(Markdown(f'Generator installation size: {lm.results["gen-size"]:.0f} kW'))
display(Markdown(f'Total length of lines is {lm.results["line-length"]:.0f}m'))
display(Markdown(f'CAPEX: ${lm.results["capex"]:.0f}'))
display(Markdown(f'Annual OPEX: ${lm.results["opex"]:.0f}'))
display(Markdown(f'Annual Income: ${lm.results["income"]:.0f}'))
display(Markdown(f'**NPV over {lm.years} years is ${lm.results["npv"]:.0f}**'))